In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd
from scipy.interpolate import interp1d

In [ ]:
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
import lsst.afw.cameraGeom.utils as cgUtils
from lsst.daf.persistence import Butler, NoResults

In [ ]:
import lsst.syseng.throughputs as st
from lsst.sims.photUtils import PhotometricParameters, Bandpass, LSSTdefaults
from lsst.sims.utils import angularSeparation

In [ ]:
#rname = 'R10'
#rname = 'R22'
#rname = 'R01'
#rname = 'R11'
#rname = 'R20'
#rname = 'R21'
rname = 'R30'
#rname = 'R12'
#rname = 'R02'
#rname = 'R31'
#rname = 'R03'
#rname = 'R34'
#rname = 'R13'
#rname = 'R23'
#rname = 'R14'
#rname = 'R32'
#rname = 'R24'
#rname = 'R41'
#rname = 'R42'
#rname = 'R33'
#rname = 'R43'

In [ ]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
useDefault = False
try:
    if np.isnan(dd.rtm[rname]):
        useDefault = True
        print('no data yet, use default')
except TypeError:
    pass

### Default photometric parameters, as used in standard m5 calculations
Note that effarea is not in this list here, because it varies with field.

The read noise is not in this list either, because it varies by amp.

In [ ]:
exptime=15 
nexp=2
othernoise=0 
darkcurrent=0.2
X=1.0

In [ ]:
#we do not need this cell to calculate m5, but these FWHMeff are the default values we use actually
lsstDefaults = LSSTdefaults()

### Set up throughputs for hardware and atmosphere. Use the default detector QE as in syseng_throughput for now

In [ ]:
# Add losses to each component?
addLosses = True
defaultDirs = st.setDefaultDirs()
defaultDirs

In [ ]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
mirror1 = st.buildMirror(defaultDirs['mirror1'], addLosses)
mirror2 = st.buildMirror(defaultDirs['mirror2'], addLosses)
mirror3 = st.buildMirror(defaultDirs['mirror3'], addLosses)
lens1 = st.buildLens(defaultDirs['lens1'], addLosses)
lens2 = st.buildLens(defaultDirs['lens2'], addLosses)
lens3 = st.buildLens(defaultDirs['lens3'], addLosses)
filters = st.buildFilters(defaultDirs['filters'], addLosses)

vendor = dd.vendor[rname]
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector0 = st.buildDetector(vendorDir, addLosses) #design QE from this vendor
detector = Bandpass()

In [ ]:
for f in filters:
    print(f, lsstDefaults.FWHMeff(f), lsstDefaults.m5(f))

### Butler access to the QE data

In [ ]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

#This is no longer needed after tickets/DM-22605
#from lsst.obs.lsst.lsstCamMapper import LsstCamMapper
#mapper = LsstCamMapper()
#lsstcam = mapper.camera

### Prepare vignetting function

In [ ]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

### Create the dataframes for this raft

In [ ]:
#filterlist = tuple([s for s in filters]+['fS']) # this would be in different order than below
filterlist = ['u', 'g', 'r', 'i', 'z', 'y', 'fS', 'u1', 'u2']
alist = ('raDeg', 'decDeg', 'radDeg', 'effarea', 'readnoise', 'gain', 'saturation')
detectors = []
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    detectors.append(det.getName())
adf = pd.DataFrame(index=alist, columns=detectors, dtype=object)
m5df = pd.DataFrame(index=filterlist, columns=detectors, dtype=object)
Tdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)
Sdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)

In [ ]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    key = rname+'_'+dname
    raDeg = {}
    decDeg = {}
    readnoise = {}
    gain = {}
    saturation = {}
    for amp in det:
        i = amp.getName()
        amp_point = amp.getBBox().getCenter()
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [raDeg[i], decDeg[i]] = np.degrees(raDec)
        gain[i] = amp.getGain()
        saturation[i] = amp.getSaturation()
        #print(key, i, raDec, amp.getGain(), amp.getSaturation())
        if useDefault:
            readnoise[i] = 8.8
        else:
            readnoise[i] = amp.getReadNoise()
    adf[key].loc['raDeg'] = list(raDeg.values())
    adf[key].loc['decDeg'] = list(decDeg.values())
    adf[key].loc['readnoise'] = list(readnoise.values())
    adf[key].loc['gain'] = list(gain.values())
    adf[key].loc['saturation'] = list(saturation.values())
    
    #effective area
    radius = angularSeparation(0., 0., adf[key]['raDeg'], adf[key]['decDeg'])
    adf[key].loc['radDeg'] = list(radius)
    adf[key].loc['effarea'] = list(np.interp(radius, vr, vv)*np.pi*(M1D/2)**2)

In [ ]:
adf['R30_S00'].loc['raDeg']

In [ ]:
adf

In [ ]:
#adf['R32_S11'].loc['readnoise']

In [ ]:
ampList = list(raDeg.keys())

In [ ]:
# this is needed if there are only 6 QE measurements per amp
idx = np.where(detector0.sb>0.01)
idx1=idx[0][0]-1
idx2=idx[0][-1]+1

x1 = detector0.wavelen[idx1]
y1 = detector0.sb[idx1]
x2 = detector0.wavelen[idx2]
y2 = detector0.sb[idx2]

In [ ]:
m5SRD = np.array([23.9, 25.0, 24.7, 24.0, 23.3, 22.1])
#m5SRDmin = []
# Nv1 from SRD table 24
Nv1 = np.array([56, 80, 184, 184, 160, 160])
omega = Nv1/sum(Nv1)
fidx = 'ugrizy' #very important!

In [ ]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
        
    vendor = det.getSerial()[:3].lower()
    assert useDefault or dd.vendor[rname].lower() == vendor
    vendorDir = defaultDirs['detector']+'/../'+vendor
    print('Calculating m5 for %s_%s'%(rname,dname))
    
    key = rname+'_'+dname
    for f in filterlist:
        m5df[key][f] = [-1.]*len(ampList)
    for f in filterlist[:6]:
        Tdf[key][f] = [-1.]*len(ampList)
        Sdf[key][f] = [-1.]*len(ampList)
        
    for amp in det:
        
        try:
            qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, calibDate='1970-01-01T00:00:00')
            wavelen = detector0.wavelen 

            k = amp.getName()
            if len(qe_curve.data[k][0])>10:
                wlen = qe_curve.data[k][0]
                eff = qe_curve.data[k][1]
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='quadratic')
            else:
                aa = np.append(x1, qe_curve.data[k][0].value)
                aa = np.append(aa, x2)
                wlen = aa * qe_curve.data[k][0].unit

                aa = np.append(y1, qe_curve.data[k][1].value)
                aa = np.append(aa, y2)
                eff = aa * qe_curve.data[k][1].unit
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='slinear')#quadratic causes overshoot
            
            sb = f(wavelen)*0.01
            #alternatively we could do (only for >10 QE measurements)
            #amp_point = amp.getBBox().getCenter()
            #sb = qe_curve.evaluate(det, amp_point, wavelen* u.nm, kind='quadratic').value*.01 #unit was percent in CALIB data

            sb[np.isnan(sb)] = 0
            if np.max(sb)>1.5:
                print('These seem too LARGE ', k)
                print(np.max(sb))
                sb = 0
            if np.max(sb)<0.2: #3 dead channels, 1 out of each of R01, R10, and R30; see camera confluence page table
                print('dead channel: %s %s, max sb = %.2f'%(key, amp.getName(), np.max(sb)))
                continue;
                
            detector.setBandpass(wavelen, sb)
                
            #detector losses  
            #os.listdir(vendorDir)
            detLosses = Bandpass()
            detLosses.readThroughput(os.path.join(vendorDir, '%s_Losses/det_Losses.dat' % (vendor)))
                
            #build hardware and system
            hardware = {}
            system = {}
            for f in filters:
                sb = mirror1.sb * mirror2.sb *mirror3.sb
                sb *= lens1.sb * lens2.sb * lens3.sb * filters[f].sb
                sb *= detector.sb * detLosses.sb
                
                hardware[f] = Bandpass()
                hardware[f].setBandpass(wavelen, sb)
                system[f] = Bandpass()
                system[f].setBandpass(wavelen, sb * atmos.sb)
                
        except NoResults:
            if not useDefault:
                print('No results found for this detector')
            assert useDefault
            
            hardware, system = st.buildHardwareAndSystem(defaultDirs)
            
        #calculate m5      
        iamp = ampList.index(amp.getName())
        effarea = adf[key]['effarea'][iamp]*100**2 #convert to cm^2
        readnoise = adf[key]['readnoise'][iamp]
        
        m5 = st.makeM5(hardware, system, darksky=None, 
                    exptime=15, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                    effarea=effarea, X=1.0)
        for f in filters:
            m5df[key][f][iamp] = m5.m5[f]
            Tdf[key][f][iamp] = m5.Tb[f]
            Sdf[key][f][iamp] = m5.Sb[f]
        m5amp = np.array([m5.m5[f] for f in fidx])
        if np.all(m5amp>0):
            m5df[key]['fS'][iamp] = sum(omega*10**(0.8*(m5amp - m5SRD)))
        
        #what about u-band with 1min & 2 min visits?
        m5 = st.makeM5(hardware, system, darksky=None, 
                    exptime=30, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                    effarea=effarea, X=1.0)
        m5df[key]['u1'][iamp] = m5.m5['u']
        m5 = st.makeM5(hardware, system, darksky=None, 
                    exptime=60, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                    effarea=effarea, X=1.0)
        m5df[key]['u2'][iamp] = m5.m5['u']        

In [ ]:
Sdf

In [ ]:
m5df

In [ ]:
dfDir = os.path.join('m5_output', rname)

In [ ]:
if not os.path.exists(dfDir):
    os.mkdir(dfDir)
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
adf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
m5df.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Tdf_%s.csv'%rname)
Tdf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Sdf_%s.csv'%rname)
Sdf.to_csv(dfPath)

### Make sure we can read out the dataframes correctly

In [ ]:
from ast import literal_eval

In [ ]:
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [ ]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

In [ ]:
adf

In [ ]:
adf['R30_S00'].loc['raDeg']

In [ ]:
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [ ]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

In [ ]:
#df['R01_S11']['u']